In [1]:
import torch
import numpy
import torch.nn as nn

In [2]:
def non_linear_transition(x):
    x1 = x[0] **2
    x2 = x[1] **3
    x3 = x[2] **4
    x4 = torch.sin(x[3])
    reward = x.sum() #Linear Reward (can be quadratic)
    return (torch.tensor([x1, x2, x3, x4]), torch.tensor([reward]))
train = []
test = []
N = 1000
for i in range(N):
    x = torch.randn(4)
    y, r = non_linear_transition(x)
    if i % 10 == 0:
        test.append((x, y,r))
    else:
        train.append((x, y, r))

In [3]:
enc_dim = 16
state_encoder = torch.nn.Sequential(
    torch.nn.Linear(4, enc_dim//2),
    torch.nn.ReLU(),
    torch.nn.Linear(enc_dim//2, enc_dim),
    torch.nn.ReLU(),
    torch.nn.Linear(enc_dim, enc_dim),
)

state_decoder = torch.nn.Sequential(
    torch.nn.Linear(enc_dim, enc_dim//2),
    torch.nn.ReLU(),
    torch.nn.Linear(enc_dim//2, 4)
)

A = torch.nn.Parameter(torch.randn(enc_dim, enc_dim))
Q = torch.nn.Parameter(torch.randn(enc_dim, enc_dim))
optimizer = torch.optim.Adam(list(state_encoder.parameters()) + list(state_decoder.parameters()) + [A] + [Q], lr=1e-3)

In [4]:
#train, test are lists of tuples of (x, y, r)
epochs = 10000
criterion = torch.nn.MSELoss()
for i in range(epochs):
    total_state_loss = 0
    total_reward_loss = 0
    for x, y, r in train:
        optimizer.zero_grad()
        xx = state_encoder(x)
        pred_state = A @ xx
        Q_psd = Q.T @ Q
        pred_reward = xx.T @ Q_psd @ xx
        dec_state = state_decoder(pred_state)   
        state_og_loss = criterion(dec_state, y)
        state_enc_loss = criterion(state_encoder(y), xx)
        auto_encode_loss = criterion(state_decoder(state_encoder(x)), x)
        state_loss = state_og_loss + state_enc_loss + auto_encode_loss
        reward_loss = criterion(pred_reward, r)
        loss = state_loss #+ reward_loss 
        loss.backward()
        optimizer.step()
        total_state_loss += state_loss.item()
        total_reward_loss += reward_loss.item()
    if i % 10 == 0:
        with torch.no_grad():
            total_test_state_loss = 0
            total_test_reward_loss = 0
            for x, y,r  in test:
                xx = state_encoder(x)
                pred_state = A @ xx
                Q_psd = Q.T @ Q
                pred_reward = xx.T @ Q_psd @ xx
                dec_state = state_decoder(pred_state)
                state_og_loss = criterion(dec_state, y)
                state_enc_loss = criterion(state_encoder(y), xx)
                auto_encode_loss = criterion(state_decoder(state_encoder(x)), x)
                state_loss = state_og_loss + state_enc_loss + auto_encode_loss
                reward_loss = criterion(pred_reward, r)
                total_test_state_loss += state_loss.item()
                total_test_reward_loss += reward_loss.item()
            print(f"Epoch {i}: Train State Loss: {total_state_loss/N}, Train Reward Loss: {total_reward_loss/N}, Test State Loss: {total_test_state_loss/len(test)}, Test Reward Loss: {total_test_reward_loss/len(test)}")

C:\Users\mikef\AppData\Local\Temp\ipykernel_8444\4282157989.py:12: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at C:\b\abs_e1nmgx0bk6\croot\pytorch-select_1725478824526\work\aten\src\ATen\native\TensorShape.cpp:3679.)
  pred_reward = xx.T @ Q_psd @ xx
c:\Users\mikef\anaconda3\envs\gym\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 0: Train State Loss: 23.948470457628368, Train Reward Loss: 337.1558699510098, Test State Loss: 30.52271986424923, Test Reward Loss: 420.91455726623536
Epoch 10: Train State Loss: 4.490297100752592, Train Reward Loss: 2839.6854546813965, Test State Loss: 5.8039762602746485, Test Reward Loss: 3056.0170069885253
Epoch 20: Train State Loss: 3.1285800026506188, Train Reward Loss: 3856.889682495117, Test State Loss: 3.9753887915611266, Test Reward Loss: 4656.601389312744
Epoch 30: Train State Loss: 2.5718757000640036, Train Reward Loss: 4279.302886932373, Test State Loss: 3.131368246078491, Test Reward Loss: 4872.341660308838
Epoch 40: Train State Loss: 2.0009902143627407, Train Reward Loss: 3165.4726855010986, Test State Loss: 2.0337191195786, Test Reward Loss: 3402.8020547485353
Epoch 50: Train State Loss: 1.7177786374166608, Train Reward Loss: 2755.3142679290772, Test State Loss: 2.85171800032258, Test Reward Loss: 2996.986884918213
Epoch 60: Train State Loss: 1.7208269734680652, T

KeyboardInterrupt: 

In [37]:
t = train[0]
x, y, r = t
xx = state_encoder(x)
pred_state = A @ xx
dec_state = state_decoder(pred_state)
print(x)
print(xx)
print(y)
print(dec_state)
print("Test")
t = test[0]
x, y,r = t
xx = state_encoder(x)
pred_state = A @ xx
dec_state = state_decoder(pred_state)
print(x)
print(y)
print(dec_state)

tensor([-0.3892, -1.1420,  0.4347,  0.3648])
tensor([ 0.0351, -0.0070,  0.1394, -1.6561,  0.3626, -0.8260,  1.1684,  0.1074,
         0.7020, -0.0528, -0.3366,  0.7623, -0.2779, -0.1570,  0.3622,  0.0454],
       grad_fn=<ViewBackward0>)
tensor([ 0.1514, -1.4894,  0.0357,  0.3568])
tensor([ 0.1218, -1.5987,  0.1711,  0.1662], grad_fn=<ViewBackward0>)
Test
tensor([ 0.6592, -0.8172, -1.5943, -2.5283])
tensor([ 0.4345, -0.5458,  6.4610, -0.5755])
tensor([ 0.3680, -0.9164,  6.7711, -1.4532], grad_fn=<ViewBackward0>)
